# 🚀 Advanced Multi-Agent System Evaluation Demo

This notebook demonstrates a complete MAS (Multi-Agent System) with:
- **Parallel Researchers** - Two researchers working simultaneously
- **Sequential Writer** - Aggregating research into a document
- **Critic Loop** - Iterative refinement with quality feedback

All agents use Google ADK with Gemini 2.5 Flash, and the full `mas_eval` evaluation framework runs on captured traces.

### Architecture
```
    ┌──────────────┐     ┌──────────────┐
    │ Researcher A │     │ Researcher B │
    │  (parallel)  │     │  (parallel)  │
    └──────┬───────┘     └──────┬───────┘
           │                     │
           └──────────┬──────────┘
                      ▼
              ┌───────────────┐
              │    Writer     │
              │ (aggregator)  │
              └───────┬───────┘
                      │
                      ▼
              ┌───────────────┐
              │    Critic     │◄─────┐
              │   (review)    │      │
              └───────┬───────┘      │
                      │              │
                      ▼              │
              ┌───────────────┐      │
              │  Revise if    │──────┘
              │   needed      │
              └───────────────┘
```

## 📦 Step 1: Installation

First, let's install all required dependencies including Google ADK.

In [ ]:
# Install required packages
!pip install -q google-adk google-genai networkx numpy sentence-transformers

## 🔑 Step 2: API Key Configuration

Set your Google API key. In Colab, you can use Secrets or set it directly.

In [ ]:
import os
from google.colab import userdata

# Option 1: Use Colab Secrets (recommended)
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    print("✅ API key loaded from Colab Secrets")
except:
    # Option 2: Set directly (not recommended for sharing)
    # os.environ['GOOGLE_API_KEY'] = 'your-api-key-here'
    print("⚠️ Please set GOOGLE_API_KEY in Colab Secrets or uncomment the line above")

## 📂 Step 3: Upload mas_eval Framework

Upload your `mas_eval` folder to Colab, or clone it from your repository.

In [ ]:
# Option 1: Upload mas_eval folder manually using the file browser on the left
# Then run:
import sys
sys.path.insert(0, '/content')

# Option 2: Clone from GitHub (if available)
# !git clone https://github.com/your-repo/mas_eval.git

# Verify mas_eval is importable
try:
    from mas_eval.adapters.adk_adapter import ADKTracingCallback
    print("✅ mas_eval framework loaded successfully!")
except ImportError as e:
    print(f"❌ Failed to import mas_eval: {e}")
    print("Please upload the mas_eval folder to /content/")

## 📚 Step 4: Import All Dependencies

In [ ]:
import asyncio
import json
from datetime import datetime
from typing import List, Dict, Any, Optional, Tuple
import uuid

# Google ADK
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# mas_eval components
from mas_eval.adapters.adk_adapter import ADKTracingCallback
from mas_eval.graph.crg_builder import CRGModule
from mas_eval.graph.agent_graph import AgentGraphBuilder, AgentGraphEvaluator
from mas_eval.graph.visualizer import GraphVisualizer
from mas_eval.mast.classifier import MASTClassifier, ClassifierMode
from mas_eval.metrics.gemmas import GEMMASMetrics
from mas_eval.metrics.thought_relevance import ThoughtRelevanceMetric
from mas_eval.suggestions.advisor import MASAdvisor
from mas_eval.evaluator import MASEvaluator
from mas_eval.core.types import Span, TraceData

print("✅ All imports successful!")

## 🤖 Step 5: Define Agent Instructions

In [ ]:
RESEARCHER_A_INSTRUCTION = """You are Researcher A, an expert in technical analysis and data gathering.

Your role:
- Research the TECHNICAL aspects of the given topic
- Focus on: implementation details, technologies, architectures, technical challenges
- Provide specific facts, statistics, and technical specifications
- Be thorough but concise

When given a topic, provide a comprehensive technical research summary.
Format your findings clearly with bullet points or numbered lists.
Always start with "TECHNICAL RESEARCH FINDINGS:" as your header.
"""

RESEARCHER_B_INSTRUCTION = """You are Researcher B, an expert in market analysis and trends.

Your role:
- Research the MARKET and BUSINESS aspects of the given topic  
- Focus on: market trends, business applications, industry adoption, economic impact
- Provide specific examples, case studies, and market data
- Be thorough but concise

When given a topic, provide a comprehensive market/business research summary.
Format your findings clearly with bullet points or numbered lists.
Always start with "MARKET RESEARCH FINDINGS:" as your header.
"""

WRITER_INSTRUCTION = """You are the Writer/Aggregator agent.

Your role:
- Combine and synthesize research from multiple sources into a coherent document
- Create a well-structured, comprehensive summary
- Ensure all key points from the research are included
- Add logical flow and transitions between sections
- Make the content accessible and professional

When given research inputs, create a unified document with:
1. Executive Summary
2. Technical Overview (from Researcher A)
3. Market Analysis (from Researcher B)  
4. Key Insights and Conclusions

Start your output with "COMPREHENSIVE REPORT:" as your header.
"""

CRITIC_INSTRUCTION = """You are the Critic agent, responsible for quality assurance.

Your role:
- Review the written document for completeness and quality
- Check for: accuracy, clarity, coherence, missing information
- Provide specific, actionable feedback
- Rate the document's quality

Provide your critique in this format:
QUALITY ASSESSMENT:
- Overall Score: [1-10]
- Completeness: [1-10]
- Clarity: [1-10]
- Coherence: [1-10]

STRENGTHS:
- [List strengths]

ISSUES TO ADDRESS:
- [List specific issues with suggestions]

VERDICT: [APPROVED / NEEDS_REVISION]

If NEEDS_REVISION, explain exactly what needs to be fixed.
Only approve if the document truly meets high quality standards.
"""

print("✅ Agent instructions defined!")

## 🏗️ Step 6: Create Agents

In [ ]:
def create_agents():
    """Create all ADK agents for the MAS."""
    model = "gemini-2.5-flash"
    
    researcher_a = Agent(
        name="ResearcherA",
        model=model,
        instruction=RESEARCHER_A_INSTRUCTION,
    )
    
    researcher_b = Agent(
        name="ResearcherB",
        model=model,
        instruction=RESEARCHER_B_INSTRUCTION,
    )
    
    writer = Agent(
        name="Writer",
        model=model,
        instruction=WRITER_INSTRUCTION,
    )
    
    critic = Agent(
        name="Critic",
        model=model,
        instruction=CRITIC_INSTRUCTION,
    )
    
    return {
        "researcher_a": researcher_a,
        "researcher_b": researcher_b,
        "writer": writer,
        "critic": critic
    }

agents = create_agents()
print(f"✅ Created {len(agents)} agents: {list(agents.keys())}")

## ⚙️ Step 7: Agent Execution Helpers

In [ ]:
async def run_agent(
    agent: Agent,
    message: str,
    callback: ADKTracingCallback,
    session_id: str = None,
    user_id: str = "user"
) -> Tuple[str, List[Span]]:
    """Run a single agent and capture spans."""
    runner = InMemoryRunner(agent=agent, app_name=f"mas-{agent.name}")
    session_id = session_id or str(uuid.uuid4())
    
    session = await runner.session_service.create_session(
        app_name=f"mas-{agent.name}",
        user_id=user_id
    )
    
    final_response = ""
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session.id,
        new_message=types.Content(
            role="user",
            parts=[types.Part(text=message)]
        )
    ):
        callback.on_event(event)
        
        if hasattr(event, 'content') and event.content:
            if hasattr(event.content, 'parts'):
                for part in event.content.parts:
                    if hasattr(part, 'text') and part.text:
                        final_response = part.text
    
    return final_response, callback.get_spans()


async def run_parallel_researchers(
    researcher_a: Agent,
    researcher_b: Agent,
    topic: str,
    callback: ADKTracingCallback
) -> Tuple[str, str]:
    """Run two researchers in parallel."""
    print("\n" + "="*60)
    print("🔬 PHASE 1: PARALLEL RESEARCH")
    print("="*60)
    print(f"📋 Topic: {topic[:100]}...")
    print("⚡ Running Researcher A and Researcher B in parallel...")
    
    callback_a = ADKTracingCallback(service_name="researcher-a", verbose=False)
    callback_b = ADKTracingCallback(service_name="researcher-b", verbose=False)
    
    task_a = run_agent(researcher_a, topic, callback_a)
    task_b = run_agent(researcher_b, topic, callback_b)
    
    results = await asyncio.gather(task_a, task_b)
    
    research_a, spans_a = results[0]
    research_b, spans_b = results[1]
    
    # Merge spans into main callback
    for span in spans_a:
        callback._spans.append(span)
        callback._span_id_set.add(span.span_id)
    for span in spans_b:
        callback._spans.append(span)
        callback._span_id_set.add(span.span_id)
    
    print(f"✅ Researcher A completed: {len(research_a)} chars")
    print(f"✅ Researcher B completed: {len(research_b)} chars")
    
    return research_a, research_b


async def run_writer(
    writer: Agent,
    research_a: str,
    research_b: str,
    callback: ADKTracingCallback
) -> str:
    """Run writer to aggregate research."""
    print("\n" + "="*60)
    print("✍️  PHASE 2: AGGREGATION & WRITING")
    print("="*60)
    print("📝 Combining research from both researchers...")
    
    writer_input = f"""Please combine and synthesize the following research into a comprehensive document:

--- RESEARCHER A (Technical Analysis) ---
{research_a}

--- RESEARCHER B (Market Analysis) ---
{research_b}

Create a unified, well-structured document that incorporates insights from both research sources.
"""
    
    writer_callback = ADKTracingCallback(service_name="writer", verbose=False)
    document, spans = await run_agent(writer, writer_input, writer_callback)
    
    for span in spans:
        callback._spans.append(span)
        callback._span_id_set.add(span.span_id)
    
    print(f"✅ Writer completed: {len(document)} chars")
    
    return document


async def run_critic_loop(
    writer: Agent,
    critic: Agent,
    initial_document: str,
    callback: ADKTracingCallback,
    max_iterations: int = 2
) -> Tuple[str, int]:
    """Run critic loop for document refinement."""
    print("\n" + "="*60)
    print("🔄 PHASE 3: CRITIC REVIEW LOOP")
    print("="*60)
    
    current_document = initial_document
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        print(f"\n--- Critic Review Iteration {iteration}/{max_iterations} ---")
        
        critic_callback = ADKTracingCallback(service_name=f"critic-{iteration}", verbose=False)
        critique, spans = await run_agent(
            critic,
            f"Please review this document:\n\n{current_document}",
            critic_callback
        )
        
        for span in spans:
            callback._spans.append(span)
            callback._span_id_set.add(span.span_id)
        
        print(f"📋 Critic feedback received ({len(critique)} chars)")
        
        if "APPROVED" in critique.upper() and "NEEDS_REVISION" not in critique.upper():
            print("✅ Document APPROVED by Critic!")
            break
        
        print("🔄 Document needs revision, running writer again...")
        
        revision_prompt = f"""Please revise this document based on the critic's feedback:

CURRENT DOCUMENT:
{current_document}

CRITIC'S FEEDBACK:
{critique}

Create an improved version addressing all the issues mentioned.
"""
        
        revision_callback = ADKTracingCallback(service_name=f"revision-{iteration}", verbose=False)
        current_document, spans = await run_agent(writer, revision_prompt, revision_callback)
        
        for span in spans:
            callback._spans.append(span)
            callback._span_id_set.add(span.span_id)
        
        print(f"📝 Revision completed ({len(current_document)} chars)")
    
    if iteration >= max_iterations:
        print(f"⚠️  Reached max iterations ({max_iterations})")
    
    return current_document, iteration

print("✅ Execution helpers defined!")

## 🚀 Step 8: Run the MAS Pipeline

In [ ]:
async def run_mas_pipeline(topic: str) -> Tuple[str, ADKTracingCallback]:
    """Run the complete MAS pipeline."""
    print("\n" + "="*70)
    print("🚀 ADVANCED MULTI-AGENT SYSTEM DEMO")
    print("="*70)
    print(f"🎯 Topic: {topic[:100]}...")
    print(f"⏰ Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    agents = create_agents()
    
    master_callback = ADKTracingCallback(
        service_name="advanced-mas",
        verbose=True,
        max_spans=10000
    )
    
    # Phase 1: Parallel Research
    research_a, research_b = await run_parallel_researchers(
        agents["researcher_a"],
        agents["researcher_b"],
        topic,
        master_callback
    )
    
    # Phase 2: Sequential Writing
    initial_document = await run_writer(
        agents["writer"],
        research_a,
        research_b,
        master_callback
    )
    
    # Phase 3: Critic Loop
    final_document, iterations = await run_critic_loop(
        agents["writer"],
        agents["critic"],
        initial_document,
        master_callback,
        max_iterations=2
    )
    
    print("\n" + "="*70)
    print("✅ MAS PIPELINE COMPLETE")
    print("="*70)
    print(f"📊 Total spans captured: {len(master_callback.get_spans())}")
    print(f"🔄 Critic iterations: {iterations}")
    
    return final_document, master_callback

In [ ]:
# Define the research topic
topic = """
Research the topic: "The Impact of Large Language Models on Software Development"

Cover both technical aspects (how LLMs work, integration methods, tooling) 
and market/business aspects (adoption rates, industry trends, economic impact).
"""

# Run the MAS pipeline
final_document, callback = await run_mas_pipeline(topic)

## 📄 Step 9: View Final Document

In [ ]:
from IPython.display import Markdown, display

print("="*70)
print("📄 FINAL DOCUMENT")
print("="*70)
display(Markdown(final_document))

## 📊 Step 10: Run Full Evaluation Framework

In [ ]:
def run_full_evaluation(
    callback: ADKTracingCallback,
    task_description: str = "Research and write a comprehensive document"
) -> Dict[str, Any]:
    """Run the complete evaluation framework on captured traces."""
    print("\n" + "="*70)
    print("📊 RUNNING FULL EVALUATION FRAMEWORK")
    print("="*70)
    
    results = {}
    spans = callback.get_spans()
    
    if not spans:
        print("⚠️  No spans captured! Cannot run evaluation.")
        return {"error": "No spans captured"}
    
    print(f"📋 Evaluating {len(spans)} spans from {len(set(s.agent_name for s in spans))} agents")
    
    # 1. CRG Construction
    print("\n" + "-"*50)
    print("1️⃣  CAUSAL REASONING GRAPH (CRG)")
    print("-"*50)
    
    crg = CRGModule()
    graph = crg.build(spans)
    crg_stats = crg.get_statistics()
    print(f"   Nodes: {crg_stats.get('num_nodes', 0)}")
    print(f"   Edges: {crg_stats.get('num_edges', 0)}")
    print(f"   Agents: {crg_stats.get('agents', [])}")
    results["crg"] = crg_stats
    
    # 2. Agent Interaction Graph
    print("\n" + "-"*50)
    print("2️⃣  AGENT INTERACTION GRAPH")
    print("-"*50)
    
    agent_graph_builder = AgentGraphBuilder()
    agent_graph = agent_graph_builder.build(spans)
    agent_evaluator = AgentGraphEvaluator()
    agent_metrics = agent_evaluator.evaluate(agent_graph, spans)
    
    print(f"   Agent Count: {agent_metrics.get('agent_count', 0)}")
    print(f"   Collaboration Score: {agent_metrics.get('overall_score', 0):.2%}")
    results["agent_graph"] = agent_metrics
    
    # 3. GEMMAS Metrics
    print("\n" + "-"*50)
    print("3️⃣  GEMMAS METRICS (IDS & UPR)")
    print("-"*50)
    
    gemmas = GEMMASMetrics()
    gemmas_metrics = gemmas.compute(graph, spans)
    
    print(f"   IDS (Information Diversity): {gemmas_metrics.get('IDS', 0):.4f}")
    print(f"   UPR (Unnecessary Path Ratio): {gemmas_metrics.get('UPR', 0):.4f}")
    results["gemmas"] = gemmas_metrics
    
    # 4. Thought Relevance
    print("\n" + "-"*50)
    print("4️⃣  THOUGHT RELEVANCE SCORING")
    print("-"*50)
    
    try:
        relevance_metric = ThoughtRelevanceMetric()
        relevance_scores = relevance_metric.compute(spans, task_description)
        print(f"   Average Relevance: {relevance_scores.get('average_relevance', 0):.4f}")
        results["thought_relevance"] = relevance_scores
    except Exception as e:
        print(f"   ⚠️  Failed: {e}")
        results["thought_relevance"] = {"error": str(e)}
    
    # 5. MAST Classification
    print("\n" + "-"*50)
    print("5️⃣  MAST FAILURE MODE CLASSIFICATION")
    print("-"*50)
    
    try:
        mast = MASTClassifier(
            mode=ClassifierMode.FEW_SHOT,
            model="gemini-2.5-flash",
            verbose=True
        )
        failure_modes = mast.classify(spans, task_description=task_description)
        
        if failure_modes:
            print(f"\n   🚨 Detected {len(failure_modes)} potential failure modes:")
            for fm in failure_modes:
                print(f"   [{fm.code}] {fm.name} - {fm.get_probability_percentage()}")
        else:
            print("   ✅ No significant failure modes detected!")
        
        results["mast"] = {
            "failure_count": len(failure_modes),
            "failures": [{
                "code": fm.code,
                "name": fm.name,
                "category": fm.category,
                "probability": fm.get_probability()
            } for fm in failure_modes]
        }
    except Exception as e:
        print(f"   ⚠️  Failed: {e}")
        results["mast"] = {"error": str(e), "failure_count": 0}
    
    # 6. MAS Improvement Suggestions
    print("\n" + "-"*50)
    print("6️⃣  MAS IMPROVEMENT SUGGESTIONS")
    print("-"*50)
    
    try:
        advisor = MASAdvisor(model="gemini-2.5-flash")
        suggestions = advisor.generate_suggestions_from_data(
            spans=spans,
            failures=[],
            metrics=results.get("gemmas", {}),
            graph_stats=results.get("crg", {}),
            task_description=task_description
        )
        
        if suggestions:
            print("\n   📝 Improvement Suggestions:")
            for i, s in enumerate(suggestions[:3], 1):
                title = s.title if hasattr(s, 'title') else s.get('title', 'Suggestion')
                print(f"   {i}. {title}")
        # Convert Suggestion objects to dicts for JSON serialization
        results["suggestions"] = [s.to_dict() if hasattr(s, 'to_dict') else s for s in suggestions[:5]] if suggestions else []
    except Exception as e:
        print(f"   ⚠️  Failed: {e}")
        results["suggestions"] = {"error": str(e)}
    
    # Calculate overall score
    ids = results.get("gemmas", {}).get("IDS", 0.5)
    upr = results.get("gemmas", {}).get("UPR", 0.5)
    collab = results.get("agent_graph", {}).get("overall_score", 0.5)
    failure_count = results.get("mast", {}).get("failure_count", 0)
    
    quality_score = (
        ids * 0.25 +
        (1 - upr) * 0.25 +
        collab * 0.25 +
        max(0, 1 - failure_count * 0.1) * 0.25
    )
    
    print("\n" + "="*70)
    print("📊 EVALUATION SUMMARY")
    print("="*70)
    print(f"   ⭐ OVERALL QUALITY SCORE: {quality_score:.2%}")
    
    results["overall_score"] = quality_score
    results["timestamp"] = datetime.now().isoformat()
    
    return results

In [ ]:
# Run the evaluation
task_desc = "Research and write a comprehensive document about LLMs in software development"
results = run_full_evaluation(callback, task_desc)

## 📈 Step 11: Visualize Results

In [ ]:
from IPython.display import HTML

def create_results_dashboard(results: Dict[str, Any]) -> str:
    """Create an HTML dashboard for results."""
    overall = results.get('overall_score', 0)
    ids = results.get('gemmas', {}).get('IDS', 0)
    upr = results.get('gemmas', {}).get('UPR', 0)
    collab = results.get('agent_graph', {}).get('overall_score', 0)
    failures = results.get('mast', {}).get('failure_count', 0)
    
    html = f"""
    <div style="font-family: system-ui; background: #1e293b; color: #f1f5f9; padding: 20px; border-radius: 16px;">
        <h2 style="color: #818cf8; margin-bottom: 20px;">📊 MAS Evaluation Dashboard</h2>
        
        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px;">
            <div style="background: #334155; padding: 15px; border-radius: 12px; text-align: center;">
                <div style="font-size: 2em; color: #22c55e;">{overall:.0%}</div>
                <div style="color: #94a3b8;">Overall Score</div>
            </div>
            <div style="background: #334155; padding: 15px; border-radius: 12px; text-align: center;">
                <div style="font-size: 2em; color: #6366f1;">{ids:.2%}</div>
                <div style="color: #94a3b8;">Info Diversity</div>
            </div>
            <div style="background: #334155; padding: 15px; border-radius: 12px; text-align: center;">
                <div style="font-size: 2em; color: #f59e0b;">{upr:.2%}</div>
                <div style="color: #94a3b8;">Path Ratio</div>
            </div>
            <div style="background: #334155; padding: 15px; border-radius: 12px; text-align: center;">
                <div style="font-size: 2em; color: #a855f7;">{collab:.2%}</div>
                <div style="color: #94a3b8;">Collaboration</div>
            </div>
            <div style="background: #334155; padding: 15px; border-radius: 12px; text-align: center;">
                <div style="font-size: 2em; color: {'#ef4444' if failures > 0 else '#22c55e'};">{failures}</div>
                <div style="color: #94a3b8;">Failures Found</div>
            </div>
        </div>
    </div>
    """
    return html

display(HTML(create_results_dashboard(results)))

## 💾 Step 12: Save Results

In [ ]:
# Save evaluation results to JSON
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"evaluation_report_{timestamp}.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, default=str)

print(f"✅ Results saved to: {output_path}")

# Download the file in Colab
from google.colab import files
files.download(output_path)

---
## 🎉 Demo Complete!

You've successfully run:
1. ✅ Parallel research with 2 agents
2. ✅ Sequential document writing
3. ✅ Iterative critic review loop
4. ✅ Full MAS evaluation with MAST classification
5. ✅ Quality metrics and improvement suggestions